In [1]:
import requests
import pandas as pd
import folium

#url = "https://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:monumenter&outputFormat=csv&SRSNAME=EPSG:4326"

#re = requests.get(url, stream=True)
#open("monuments.csv", 'wb+').write(re.content)

filename = "monuments.csv"

data1 = pd.read_csv("monuments.csv", sep=",")

In [7]:
# 1. Hvor mange monumenter er der i københavn?
print("Der er",len(data1), "Monumenter i københavn")

Der er 389 Monumenter i københavn


In [8]:
# 2. Hvor mange monumenter bliver vedligeholdt? Dvs. graffitirenhold = ja
vedligeholdtMonument = data1[(data1['graffitirenhold'] == 'Ja')]
print("Der er",len(vedligeholdtMonument),"vedligeholdte monumenter")

Der er 255 vedligeholdte monumenter


In [9]:
# Tegner københavn området & Init kortet (map_osm)
def resetMap():
    url_environ = 'http://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:miljozone&outputFormat=json&SRSNAME=EPSG:4326'
    geo_json = get_geojson(url_environ)
    map_osm = folium.Map(location=[55.686724,12.570072], zoom_start=11)
    folium.GeoJson(geo_json, name='geojson').add_to(map_osm)
    return map_osm


def monumentById(monumentId):
    monumentWithId = data1[(data1['id'] == monumentId)]
    longitude = monumentWithId["longitude"].values
    latitude = monumentWithId["latitude"].values
    return longitude, latitude


def monumentByCoords(x,y):
    monument = data1[(data1['x'] == x) & (data1['y'] == y)]
    name = monument["navn"].values[0]
    return name


# henter københavn området
def get_geojson(url): 
    response = requests.get(url)
    geo_json = response.json()
    return geo_json


# Sætter en marker på map
def drawAndPlot(pointToDraw):
    map_osm = resetMap()
    folium.Marker(location=[pointToDraw[1],pointToDraw[0]]).add_to(map_osm)
    return map_osm
    

def drawAllMonuments():
    map_osm = resetMap()
    for index, monument in data1.iterrows():
        folium.Marker(location=[monument["latitude"],monument["longitude"]]).add_to(map_osm)
    return map_osm
        
        
def interActivePlots():
    map_osm = resetMap()
    
    for index, monument in data1.iterrows():
        info = monumentByLatLot(monument["latitude"],monument["longitude"])
        ids = info[0]
        names = info[1]
        vedligeholdelse = info[2]
        iframe = folium.IFrame("Monument ID = " + str(ids) + "<br>" + "Monument name = " + str(names) + "<br>" + "Vedligeholdelse = " + str(vedligeholdelse))
        folium.Marker(location=[monument["latitude"],monument["longitude"]], popup=folium.Popup(iframe, min_width=500, max_width=500), tooltip='Click for info').add_to(map_osm)

    return map_osm


def interActivePlot(pointToDraw):
    map_osm = resetMap()    
    info = monumentByLatLot(pointToDraw[1][0],pointToDraw[0][0])
    ids = info[0]
    names = info[1]
    vedligeholdelse = info[2]
    iframe = folium.IFrame("Monument ID = " + str(ids) + "<br>" + "Monument name = " + str(names) + "<br>" + "Vedligeholdelse = " + str(vedligeholdelse))
    folium.Marker(location=[pointToDraw[1][0],pointToDraw[0][0]], popup=folium.Popup(iframe, min_width=500, max_width=500), tooltip='Click for info').add_to(map_osm)

    return map_osm



def monumentByLatLot(lat,lot):
    monument = data1[(data1['latitude'] == lat) & (data1['longitude'] == lot)]
    monId = monument["id"].values[0]
    monName = monument["navn"].values[0]
    vedligeholdt = monument["graffitirenhold"].values[0]
    return monId, monName, vedligeholdt


In [10]:
#3.a Vis monument som bliver returneret i metoden, på et kort over københavn ved brug af plotting. (Se afsnittet om 'Folium and Bokeh' under notebooks/03-3 Plotting)

drawAndPlot(monumentById(51824))


In [11]:
# 4. Find navnet på monumentet med x og y koordinaterne eller længde- og breddegraderne?

name = monumentByCoords(724712.263534,6179443.037248)
print(name)


Efterår


In [12]:
# 5. Lav en metode der optegner alle monumenterne på kortet ved brug af plotting.
drawAllMonuments()

In [13]:
# 6. Gør kortet interaktiv så navnet på monumenterne vises når man trykker på et plot. (Se afsnittet om 'Interactive plots with bokeh' under notebooks/03-3 Plotting)"

interActivePlots()

In [14]:
interActivePlot(monumentById(51824))